In [1]:
def multiply_list(lst):
    out = [None] * (2*len(lst))
    out[::2] = lst
    out[1::2] = lst
    return out

def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.express as px
    import random,pickle
    import string 
    
# Storing the sets of punctuation in variable result 
    puncsets = string.punctuation 
#     import plotly
    
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    data_df=data_df.loc[data_df['Location'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
    if len(user_df)>0:
        with open('/home/jupyter/sneupane/MOODS/pickled_files/location_abbr.pickle', 'rb') as handle:
            location_lemma = pickle.load(handle)
        y_modified_labels=location_lemma[user_id]
        cats = [ 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']      
        
        user_df.sort_values(['week', 'Location'], inplace = True)
        grps = user_df.groupby(['week', 'Location'])
        week_new = []
        for grp, df_grp in grps:
            if len(df_grp) == 1:
                week_new.append(grp[0])
            else:
                mid = len(df_grp) // 2
                prev = 0
                prev1= 0
                lower=mid
                upper=len(df_grp)-mid -1 
                for i in range(len(df_grp)):

                    if i > mid:
        #                 bias = np.random.uniform(.01, .25)
                        bias = prev + (0.2/upper) 
                        data = grp[0] + bias
                        week_new.append(data)
                        prev = bias 
                    elif i == mid:
                        week_new.append(grp[0])
                    else:
        #                 bias = np.random.uniform(.01, .25)
                        bias = prev1 + (0.2/lower) 
                        data = grp[0] - bias
                        week_new.append(data)
                        prev1 = bias 

        user_df['week_new'] = week_new
        stressors_list=user_df.Stressor.unique().tolist()
        user_df["Location_abr"]=user_df["Location"].map(y_modified_labels)
        user_df_total_group=user_df.groupby(['Location'], as_index=False)['counter'].count()
        
        locations=user_df_total_group.sort_values("counter",ascending=True)["Location"].unique().tolist()
        locations_abr=[y_modified_labels[i] for i in locations]
        stressors_list=user_df.Stressor.unique().tolist()
        if len(stressors_list) > len(colors):
            colors=multiply_list(colors)
        fig = go.Figure()
        button_list=[]
        for i in user_df.Stressor.unique():
            temp_df=user_df.loc[user_df["Stressor"]==i]
        #             print(temp_df)
            temp_df=temp_df.sort_values(["week_new"])
            fig.add_trace(go.Scatter(x= temp_df["week_new"], y=temp_df["Location_abr"],mode="markers",text=i,customdata=temp_df[["Location","week","Stressor"]],
                                 hovertemplate=  "Location : %{customdata[0]}<br>"+
                                 "Week:%{customdata[1]}<br>" +                                  
                            "Stressor: %{customdata[2]}<br>" + 
                                   
        #                                  "# of occurences: %{customdata[3]}<br>" +        
                            "<extra></extra>",opacity=1,
        #                                      visible='legendonly',
                                 name=str(i),
                                 marker=dict(size=5,color=colors[stressors_list.index(i)])
                                ))
        
       



        fig.update_layout(
            legend=dict(itemsizing= 'constant',itemclick="toggleothers",itemdoubleclick="toggle",orientation="h", y=1.1,xanchor="right",x=1,yanchor="bottom",font_size=10,
                       title_font_family="Times New Roman",title_font_size=12,
        font=dict(
            family="Courier",
            size=10,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2,),
            autosize=True,showlegend=True, 
            font=dict(

                size=10,
                color="Black"
            ),
            )
        fig.update_xaxes(range=[0,int(week_num)+0.5],nticks=int(week_num)+1,dtick=1,title="Week #")
        fig.update_yaxes( showticklabels=True, visible=True,tickangle = 270,showgrid=True,ticks="outside",ticklen=1)
        
        modified_tick_text=[]
        for items in locations:
            if len(items)>10:
                modified_tick_text.append(y_modified_labels[items])
            else:
                modified_tick_text.append(items)

        fig.update_yaxes(automargin=True,categoryorder="array",categoryarray=locations_abr)
        fig.update_layout(
                    autosize=True,legend_title="<b>Daily Stressors</b>",legend_title_side="top",

                    margin=dict(
                        l=20,
                        r=20,
                        b=20,
                        t=40,
                       
                    ),
        title={"text":"",
            "x":0.5,
             "yanchor":"top",
        "xanchor":"center",
            
           
        })
     
        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=16,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                ),
                          font=dict(
        family="Courier New, monospace",
        size=10,
        color="Black"
    ))
        data_summary_user=data_summary_user_current["Location Trend"]
        user_df_this_week=user_df.loc[user_df["week"] == week_num]
        user_df_prev_week=user_df.loc[user_df["week"] != week_num]
        if len(user_df_this_week) > 0:
            user_df_this_week_group=user_df_this_week.groupby(["Location"],as_index=False)["counter"].count().sort_values(["counter"],ascending=False)
            max_loc=user_df_this_week_group["counter"].max()
            loc_this_week=user_df_this_week_group.loc[user_df_this_week_group["counter"]==max_loc]["Location"].values.tolist()
            data_summary_user=data_summary_user.replace("_a_", str(" ,".join(loc_this_week)))
        else:
            data_summary_user=data_summary_user.replace("_a_", str("No stressors reported this week"))
        if len(user_df_prev_week) > 0:
            user_df_prev_week_group=user_df_prev_week.groupby(["Location"],as_index=False)["counter"].count().sort_values(["counter"],ascending=False)
            max_loc=user_df_prev_week_group["counter"].max()
            loc_prev_week=user_df_prev_week_group.loc[user_df_prev_week_group["counter"]==max_loc]["Location"].values.tolist()

            data_summary_user=data_summary_user.replace("_b_",  str(" ,".join(loc_prev_week)))
        data_summary_user=data_summary_user.replace("week_num", str(week_num))
       
        return "Location Trend",fig,data_summary_user
    else:
        return "","",""